In [22]:
import random
import requests

class JogoDeForca:
     def __init__(self):
         url = 'https://www.ime.usp.br/~pf/dicios/br-sem-acentos.txt'
         r = requests.get(url, allow_redirects=True)
         if r.status_code==200:
             self.content = str(r.content.decode()).split('\n')
         else:
             print("Erro: ", r.status_code)
     
     def novo_jogo(self, vidas=5):
         self.vidas = vidas
         self.palavra = random.choice(self.content)
         return len(self.palavra)

     def tentar_letra(self, letra):
         if self.vidas > 0:
             if letra in self.palavra:
                 return [idx for idx in range(len(self.palavra)) if self.palavra[idx]==letra]
             else:
                 self.vidas -= 1
                 if self.vidas == 0:
                     print("Fim de jogo!")
                     return False
                 else:
                     return []

     def tentar_palavra(self, palavra):
         if self.vidas > 0:
             if self.palavra == palavra:
                 print ("Ganhou!")
                 return True
             else:
                 self.vidas = 0
                 print("Fim de jogo!")
                 return False

jogo = JogoDeForca()
print(jogo.novo_jogo())
print(jogo.palavra)
print(jogo.vidas)
print(jogo.tentar_letra('b'))
print(jogo.tentar_letra('a'))
print(jogo.tentar_letra('w'))
print(jogo.vidas)

12
operacionais
5
[]
[4, 9]
[]
3


In [ ]:
class Jogador:
    def __init__(self, jogo):
        self.jogo = jogo
        self.df = {"vitorias": 0, "derrotas": 0}
        self.vitorias = 0
        self.derrotas = 0
        self.historico_erros = []  

    def palavras(self, tamanho, acertos, erros):
        # Aqui o que estou fazendo é basicamente filtrar as palavras com base no tamanho da palavra que foi escolhido pelo jogo/forca. Facilitando o processo de achar, além de filtramos
        # a partir dessas novas palavras XD
        palavras = [i for i in self.jogo.content if len(i) == tamanho]

        filtro = []
        for j in palavras:
            correto = True
            for i in range(len(acertos)):
                if acertos[i] != "_" and j[i] != acertos[i]:
                    correto = False
                    break
            if correto:
                filtro.append(j)
        
        palavras = filtro

        # Aqui estou garantindo que as palavras que ainda estao restando tenham letra na lista de erros, sendo assim, u filtro novamente para pegar apenas aquelas palavras cujaas letras
        # nao aparecem nos erros de caracteres. >:(
        filtro = []
        for l in palavras:
            if all(letra not in l for letra in erros):
                filtro.append(l)
        
        return filtro
    
    def freq(self, palavras, tentativas):
        # calcula a frequencia das letras que estao sobrando nas palavras restantes!
        freq = {}
        for j in palavras:
            for l in j:
                if l not in tentativas:
                    freq[l] = freq.get(l, 0) + 1
                    
        # fazendo a lista e frequencia ficar decrescente 
        freq = sorted(freq, key=freq.get, reverse=True)
        return freq
    
    def jogadas(self, jogadas):
        for _ in range(jogadas):
            tam = self.jogo.novo_jogo()
            acertos = ['_' for _ in range(tam)] # vai criar uma string do tipo, se a palavra for BANANA vai ficar '______'
            tentadas = set()
            erros = set()
            ganhou = False
            historico_jogo = []  

            while self.jogo.vidas > 0:
                palavras = self.palavras(tam, acertos, erros)
                
                # cabou palavra, cabou o codigo
                if not palavras:
                    break  

                letras = self.freq(palavras, tentadas)
                
                # cabou letra, cabou o codigo
                if not letras:
                    break  

                # olha o decrescente em acao, dessa forma posso passar como [0] bem melhor que pegar o ultimo. Enfim, pego a letra de maior prob!
                letra = letras[0]  
                tentadas.add(letra)

                res = self.jogo.tentar_letra(letra)
                
                historico_jogo.append((letra, ''.join(acertos), self.jogo.palavra, self.jogo.vidas))

                if res:
                    for i in res:
                        acertos[i] = letra
                    if "_" not in acertos:
                        ganhou = True
                        break
                else:
                    erros.add(letra)

            if ganhou:
                self.vitorias += 1
                self.df["vitorias"] = self.vitorias
            else:
                self.derrotas += 1
                self.df["derrotas"] = self.derrotas

                # :(
                self.historico_erros.append(historico_jogo)  
                # X(

    def graficos(self):
        data = pd.DataFrame.from_dict(self.df, orient='index', columns=['count']).reset_index()
        fig = go.Figure([go.Bar(x=data['index'], y=data['count'])])
        fig.update_layout(title="vic X derrotas", xaxis_title="resultados", yaxis_title="cont")
        fig.show()

    def exibir_erros(self):
        # faz o relatorio dos erros!!!!
        for idx, jogo in enumerate(self.historico_erros, 1):
            print(f"------------------------------")
            print(f"ERRO {idx}")
            for tentativa in jogo:
                letra, jogador_estado, palavra, vidas = tentativa
                print(f"{letra} | {jogador_estado} | {palavra} | {vidas}")

In [24]:
jogo = JogoDeForca()
jogador = Jogador(jogo)
jogador.jogadas(jogadas=100)
jogador.graficos()
jogador.exibir_erros()

Fim de jogo!
Fim de jogo!
Fim de jogo!
Fim de jogo!
Fim de jogo!


------------------------------
ERRO 1
a | __________ | refutaremo | 5
s | _____a____ | refutaremo | 4
r | _____a____ | refutaremo | 4
e | r____ar___ | refutaremo | 4
o | re___are__ | refutaremo | 4
m | re___are_o | refutaremo | 4
l | re___aremo | refutaremo | 3
i | re___aremo | refutaremo | 2
c | re___aremo | refutaremo | 1
g | re___aremo | refutaremo | 0
------------------------------
ERRO 2
a | ______ | magoai | 5
s | _a__a_ | magoai | 4
r | _a__a_ | magoai | 3
i | _a__a_ | magoai | 3
t | _a__ai | magoai | 2
n | _a__ai | magoai | 1
l | _a__ai | magoai | 0
------------------------------
ERRO 3
a | __________ | enviuvarem | 5
r | ______a___ | enviuvarem | 5
e | ______ar__ | enviuvarem | 5
s | e_____are_ | enviuvarem | 4
m | e_____are_ | enviuvarem | 4
n | e_____arem | enviuvarem | 4
t | en____arem | enviuvarem | 3
c | en____arem | enviuvarem | 2
x | en____arem | enviuvarem | 1
l | en____arem | enviuvarem | 0
------------------------------
ERRO 4
a | ________ | peidamos | 5
r | ____a___